In [98]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import ttest_ind
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize
from scipy.stats import boxcox
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.datasets import load_files

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from statsmodels.tools.eval_measures import mse, rmse
from wordcloud import WordCloud
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.svm import SVC
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn import neighbors
from IPython.display import Image


import pydotplus
from sklearn import ensemble

import warnings

%matplotlib inline
sns.set()

warnings.filterwarnings('ignore')
import time
import cv2
import glob

In [99]:
app_events_df = (pd.read_csv('Data/talkingdata/app_events.csv'))

app_labels_df = (pd.read_csv('Data/talkingdata/app_labels.csv'))

events_df = (pd.read_csv('Data/talkingdata/events.csv'))

gender_age_test_df = (pd.read_csv('Data/talkingdata/gender_age_test.csv'))

gender_age_train_df = (pd.read_csv('Data/talkingdata/gender_age_train.csv'))

label_categories_df = (pd.read_csv('Data/talkingdata/label_categories.csv'))

phone_brand_device_model_df = (pd.read_csv('Data/talkingdata/phone_brand_device_model.csv'))

sample_submission_df = (pd.read_csv('Data/talkingdata/sample_submission.csv'))

In [100]:
df_app_events = app_events_df
df_app_labels = app_labels_df
df_events = events_df
df_gender_age_test = gender_age_test_df
df_gender_age_train = gender_age_train_df
df_label_categories = label_categories_df

## Data Exploration

In [20]:
print(
#    app_events_df.head(),
    app_events_df.info(),
#    app_events_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32473067 entries, 0 to 32473066
Data columns (total 4 columns):
event_id        int64
app_id          int64
is_installed    int64
is_active       int64
dtypes: int64(4)
memory usage: 991.0 MB
None


In [155]:
#app_events_df['event_id'].value_counts()

#NTS: turn event_id and app_id into top 20 or so bar graphs (or histograms?) (line graph?)

In [156]:
print(
    app_labels_df.head(10),
    app_labels_df.info(),
#    app_labels_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459943 entries, 0 to 459942
Data columns (total 2 columns):
app_id      459943 non-null int64
label_id    459943 non-null int64
dtypes: int64(2)
memory usage: 7.0 MB
                app_id  label_id
0  7324884708820027918       251
1 -4494216993218550286       251
2  6058196446775239644       406
3  6058196446775239644       407
4  8694625920731541625       406
5  8694625920731541625       407
6  1977658975649789753       406
7  1977658975649789753       407
8  7311663864768030840       256
9  5902120154267999338       256 None


In [23]:
#app_labels_df['label_id'].value_counts()

In [24]:
print(
#    events_df.head(),
    events_df.info(),
#    events_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 5 columns):
event_id     int64
device_id    int64
timestamp    object
longitude    float64
latitude     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 124.1+ MB
None


In [157]:
#events_df['event_id'].value_counts()

In [26]:
print(
#    gender_age_train_df.head(),
    gender_age_train_df.info(),
#    gender_age_train_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
device_id    74645 non-null int64
gender       74645 non-null object
age          74645 non-null int64
group        74645 non-null object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB
None


In [158]:
#gender_age_train_df['device_id'].value_counts()

In [28]:
print(
#    gender_age_test_df.head(),
    gender_age_test_df.info(),
#    gender_age_test_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112071 entries, 0 to 112070
Data columns (total 1 columns):
device_id    112071 non-null int64
dtypes: int64(1)
memory usage: 875.6 KB
None


In [29]:
print(
#    label_categories_df.head(),
    label_categories_df.info(),
#    label_categories_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
label_id    930 non-null int64
category    927 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB
None


In [30]:
print(
#    phone_brand_device_model_df.head(),
    phone_brand_device_model_df.info(),
#    phone_brand_device_model_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187245 entries, 0 to 187244
Data columns (total 3 columns):
device_id       187245 non-null int64
phone_brand     187245 non-null object
device_model    187245 non-null object
dtypes: int64(1), object(2)
memory usage: 4.3+ MB
None


## Feature Engineering

In [102]:
#merging
df_merge_gender_age_train__events = pd.merge(df_gender_age_train, df_events)

In [105]:
df_merge_gender_age_train__events__app_events = pd.merge(df_merge_gender_age_train__events, df_app_events)

In [108]:
df_total_merge = pd.merge(df_merge_gender_age_train__events__app_events, df_app_labels)

In [159]:
df_total_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78844332 entries, 0 to 78844331
Data columns (total 12 columns):
device_id       int64
gender          object
age             int64
group           object
event_id        int64
timestamp       object
longitude       float64
latitude        float64
app_id          int64
is_installed    int64
is_active       int64
label_id        int64
dtypes: float64(2), int64(7), object(3)
memory usage: 7.6+ GB


In [147]:
#X will end up as the feature set with each label as a feature

X = gender_age_train_df.drop(['gender', 'age'], axis=1)

labels = app_labels_df['label_id'].value_counts().index.tolist()

for label in labels:
    X[label] = 0

In [149]:
X.describe()

,device_id,548,405,794,795,704,714,713,854,710,...,246,244,265,199,921,155,270,696,48,146
count,7.464500e+04,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,...,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0,74645.0
mean,-7.491354e+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,5.327150e+18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,-9.223067e+18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,-4.617367e+18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,-1.841362e+16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,4.636656e+18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,9.222849e+18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
count_n = 0

for device in df_total_merge['device_id']:
    label = df_total_merge['label_id'][count_n]
    X.loc[X['device_id'] == device, [label]] += 1
    count_n += 1
    